# Set up environment and load in data

In [ ]:
library(tidyverse)
library(here)
theme_set(theme_bw())
helpers_path = paste0(here(),'/analysis/helpers/')
source(paste0(helpers_path, '01_clean_behavioral_data.R'))
helpers_path = paste0(here(),'/analysis/helpers/')
source(paste0(helpers_path, '03_stim_img_helpers.R'))
fig_out_path = paste0(here(), '/outputs/fig/')

Add columns that will be used for plots below.

In [ ]:
data_yn_clean = data_yn_clean %>%
  mutate(correct = ifelse(possiblePayoff>reference & yesChosen == 1, 1, ifelse(possiblePayoff < reference & yesChosen == 0, 1, 0))) %>%
  mutate(type = ifelse(type == 1, "HT", "RE"),
         week = ifelse(week == 1, "Week 1", ifelse(week == 2, "Week 2", ifelse(week == 3, "Week 3", NA))),
         week = factor(week, levels = c("Week 1", "Week 2", "Week 3"))) %>%
  mutate(val_diff = possiblePayoff - reference,
         val_diff_bin = round(val_diff/50),
         val_diff_bin_str = paste0(val_diff_bin*50-25,":",val_diff_bin*50+25),
         val_diff_bin_str = factor(val_diff_bin_str, levels = c("-225:-175", "-175:-125", "-125:-75", "-75:-25", "-25:25", "25:75", "75:125", "125:175", "175:225")))

data_bc_clean = data_bc_clean %>%
  mutate(correct = ifelse(possiblePayoffleft>possiblePayoffright & leftChosen == 1, 1, ifelse(possiblePayoffleft<possiblePayoffright & leftChosen == 0, 1, 0))) %>%
  mutate(type = ifelse(typeLeft == 1, "HT", "RE"),
         week = ifelse(week == 1, "Week 1", ifelse(week == 2, "Week 2", ifelse(week == 3, "Week 3", NA))),
         week = factor(week, levels = c("Week 1", "Week 2", "Week 3"))) %>%
  mutate(val_diff = possiblePayoffleft - possiblePayoffright,
         val_diff_bin = round(val_diff/50),
         val_diff_bin_str = paste0(val_diff_bin*50-25,":",val_diff_bin*50+25),
         val_diff_bin_str = factor(val_diff_bin_str, levels = c("-275:-225","-225:-175", "-175:-125", "-125:-75", "-75:-25", "-25:25", "25:75", "75:125", "125:175", "175:225", "225:275"))) %>%
  filter(!is.na(val_diff_bin_str))

Center variables!

In [ ]:
data_yn_clean = data_yn_clean %>%
  mutate(day_std = (day - mean(day))/sd(day)) %>%
  group_by(subnum) %>%
  mutate(val_diff_std = (val_diff - mean(val_diff))/sd(val_diff),
         abs_val_diff = abs(val_diff),
         abs_val_diff_std = (abs_val_diff - mean(abs_val_diff))/sd(abs_val_diff))

In [ ]:
data_bc_clean = data_bc_clean %>%
  mutate(day_std = (day - mean(day))/sd(day)) %>%
  group_by(subnum) %>%
  mutate(val_diff_std = (val_diff - mean(val_diff))/sd(val_diff),
         abs_val_diff = abs(val_diff),
         abs_val_diff_std = (abs_val_diff - mean(abs_val_diff))/sd(abs_val_diff))

# HT accuracy over days plots

In [ ]:
subs = unique(data_yn_clean$subnum)

plt_data = data_yn_clean %>%
  mutate(ht_stim_num = ifelse(type == "HT", stimNum, 0)) %>%
  filter(ht_stim_num != 0) %>%
  group_by(day, subnum, ht_stim_num) %>%
  summarise(mean_accuracy = mean(correct),
            sem_accuracy = sd(correct)/sqrt(n()),
            .groups = 'keep') %>%
  ungroup()  %>%
  group_by(subnum, day) %>%
  mutate(stim_count = 1:n(),
         stim_count = as.factor(stim_count))

for(i in 1:length(subs)){
  cur_sub = subs[i]
  
  p = plt_data %>%
    filter(subnum == cur_sub) %>%
    ggplot(aes(day, mean_accuracy, shape = stim_count))+
    geom_point()+
    geom_errorbar(aes(ymin = mean_accuracy - sem_accuracy, ymax = mean_accuracy + sem_accuracy), width = 0)+
    facet_wrap(~ht_stim_num) + 
    guides(shape = "none") + 
    scale_shape_manual(values = c(1:12))+
    labs(x = "Day", y = "Accuracy", title = paste0("Y/N Task sub-", cur_sub," HT stim accuracy"))+
    theme(panel.grid = element_blank())+
    scale_x_continuous(breaks=seq(1,11,1))
  
  # Annotate plot with stim images
  out = ggplot_build(p)
  
  num_panels = length(levels(out$data[[1]]$PANEL))
  
  img_path = paste0(fig_out_path, 'stims/sub-', cur_sub, '/')
  
  for (i in 1:num_panels){
    
    cur_ht_stim_num = out$layout$layout$ht_stim_num[i]
    
    img_fn = paste0(img_path, cur_ht_stim_num, '_ht.png')
    
    cur_ht_stim_img = img_read_border_annotate(img_fn, annot_img = F, border_img = F)
    
    tmp = plt_data %>%
      filter((subnum == cur_sub) & (ht_stim_num == cur_ht_stim_num))
    
    p = p + annotation_custom2(cur_ht_stim_img, data=tmp[1,], xmin=9, xmax=11, ymin=.6, ymax=.8)
    
  }
  
  ggsave(file=paste0(fig_out_path, 'yn_sub-',cur_sub,'_HTaccuracyOverDays.jpg'), p, height = 8, width=12, units="in")
}

rm(plt_data, p)

# HT RT over days plots

In [ ]:
plt_data = data_yn_clean %>%
  mutate(ht_stim_num = ifelse(type == "HT", stimNum, 0)) %>%
  filter(ht_stim_num != 0) %>%
  group_by(day, subnum, ht_stim_num) %>%
  summarise(mean_y = mean(log(rt)),
            sem_y = sd(log(rt))/sqrt(n()),
            .groups = 'keep') %>%
  ungroup()  %>%
  group_by(subnum, day) %>%
  mutate(stim_count = 1:n(),
         stim_count = as.factor(stim_count))

for(i in 1:length(subs)){
  cur_sub = subs[i]
  
  p = plt_data %>%
    filter(subnum == cur_sub) %>%
    ggplot(aes(day, mean_y, shape = stim_count))+
    geom_point()+
    geom_errorbar(aes(ymin = mean_y - sem_y, ymax = mean_y + sem_y), width = 0)+
    facet_wrap(~ht_stim_num) + 
    guides(shape = "none") + 
    scale_shape_manual(values = c(1:12))+
    labs(x = "Day", y = "Mean Log RT", title = paste0("Y/N Task sub-", cur_sub," HT stim mean log RT"))+
    theme(panel.grid = element_blank())+
    scale_x_continuous(breaks=seq(1,11,1))
  
  # Annotate plot with stim images
  out = ggplot_build(p)
  
  num_panels = length(levels(out$data[[1]]$PANEL))
  
  img_path = paste0(fig_out_path, 'stims/sub-', cur_sub, '/')
  
  for (i in 1:num_panels){
    
    cur_ht_stim_num = out$layout$layout$ht_stim_num[i]
    
    img_fn = paste0(img_path, cur_ht_stim_num, '_ht.png')
    
    cur_ht_stim_img = img_read_border_annotate(img_fn, annot_img = F, border_img = F)
    
    tmp = plt_data %>%
      filter((subnum == cur_sub) & (ht_stim_num == cur_ht_stim_num))
    
    p = p + annotation_custom2(cur_ht_stim_img, data=tmp[1,], xmin=9, xmax=11, ymin=0, ymax=.2)
  }
  
  ggsave(file=paste0(fig_out_path, 'yn_sub-',cur_sub,'_HTmeanLogRTOverDays.jpg'), p, height = 8, width=12, units="in")  
}

rm(plt_data, p)